In [1]:
import pandas as pd
import os
import json
import re
os.getcwd()
from bs4 import BeautifulSoup

In [2]:
folder = "guideline_annotation"
guideline_list = []
haplotype_list = []
term_list = []
drug_list = []
gene_list = []
guideline_name_list = []

for path in os.listdir(folder):
    if path.endswith(".json") and "Annotation_of_" in path:
        with open(os.path.join(folder, path), "r", encoding="utf-8") as f:
            guideline_dict = json.load(f)

        path = path.replace("Annotation_of_", "").replace(".json", "")
        try:
            guideline, drug_gene = path.split("_for_")
        except:
            continue
        drugs, genes = drug_gene.split("_and_")
        drugs = list(filter(lambda x: x!= "", [x.strip() for x in drugs.split("_")]))

        tmp_haplotype_list = []
        tmp_term_list = []
        for guideline_gene in guideline_dict["guideline"]["guidelineGenes"]:
            for allele in guideline_gene["alleles"]:
                if "haplotype" in allele.keys():
                    tmp_haplotype_list.append(allele["haplotype"]["symbol"])
                    tmp_term_list.append(allele["function"]["term"])
        
        guideline_name = guideline_dict["guideline"]["name"]
        
        guideline_name_list.extend([guideline_name] * len(tmp_haplotype_list) * len(drugs))
        guideline_list.extend([guideline] * len(tmp_haplotype_list) * len(drugs))
        gene_list.extend([genes] * len(tmp_haplotype_list) * len(drugs))
        # data alignment
        for drug in drugs:
            drug_list.extend([drug] * len(tmp_haplotype_list))
            haplotype_list.extend(tmp_haplotype_list)
            term_list.extend(tmp_term_list)
        print(path)
        

df_guideline = pd.DataFrame({
    "guideline_institute": guideline_list,
    "guideline_name": guideline_name_list,
    "drug": drug_list,
    "gene": gene_list,
    "haplotype": haplotype_list,
    "term": term_list
})

print(df_guideline.shape)
df_guideline[:10]

CPIC_Guideline_for_abacavir_and_HLA_B
CPIC_Guideline_for_aceclofenac_aspirin_diclofenac_dipyrone_indomethacin_lumiracoxib_nabumetone_naproxen_and_CYP2C9
CPIC_Guideline_for_alfentanil_buprenorphine_codeine_fentanyl_hydrocodone_hydromorphone_levomethadone_morphine_naltrexone_remifentanil_sufentanil_tramadol_and_COMT_OPRM1
CPIC_Guideline_for_allopurinol_and_HLA_B
CPIC_Guideline_for_amikacin_gentamicin_kanamycin_paromomycin_plazomicin_streptomycin_tobramycin_and_MT_RNR1
CPIC_Guideline_for_amitriptyline_and_CYP2C19_CYP2D6
CPIC_Guideline_for_atazanavir_and_UGT1A1
CPIC_Guideline_for_atomoxetine_and_CYP2D6
CPIC_Guideline_for_azathioprine_and_NUDT15_TPMT
CPIC_Guideline_for_capecitabine_and_DPYD
CPIC_Guideline_for_carbamazepine_and_HLA_A_HLA_B
CPIC_Guideline_for_celecoxib_flurbiprofen_ibuprofen_lornoxicam_and_CYP2C9
CPIC_Guideline_for_citalopram_escitalopram_and_CYP2C19
CPIC_Guideline_for_clomipramine_and_CYP2C19_CYP2D6
CPIC_Guideline_for_clopidogrel_and_CYP2C19
CPIC_Guideline_for_codeine_and_CY

,guideline_institute,guideline_name,drug,gene,haplotype,term
0,CPIC_Guideline,Annotation of CPIC Guideline for abacavir and ...,abacavir,HLA_B,HLA-B*57:01:01,Presence
1,CPIC_Guideline,Annotation of CPIC Guideline for allopurinol a...,allopurinol,HLA_B,HLA-B*58:01,Presence
2,CPIC_Guideline,"Annotation of CPIC Guideline for amikacin, gen...",amikacin,MT_RNR1,MT-RNR1 663A>G,Uncertain risk of aminoglycoside-induced heari...
3,CPIC_Guideline,"Annotation of CPIC Guideline for amikacin, gen...",amikacin,MT_RNR1,MT-RNR1 669T>C,Uncertain risk of aminoglycoside-induced heari...
4,CPIC_Guideline,"Annotation of CPIC Guideline for amikacin, gen...",amikacin,MT_RNR1,MT-RNR1 747A>G,Uncertain risk of aminoglycoside-induced heari...
5,CPIC_Guideline,"Annotation of CPIC Guideline for amikacin, gen...",amikacin,MT_RNR1,MT-RNR1 786G>A,Uncertain risk of aminoglycoside-induced heari...
6,CPIC_Guideline,"Annotation of CPIC Guideline for amikacin, gen...",amikacin,MT_RNR1,MT-RNR1 807A>C,Uncertain risk of aminoglycoside-induced heari...
7,CPIC_Guideline,"Annotation of CPIC Guideline for amikacin, gen...",amikacin,MT_RNR1,MT-RNR1 807A>G,Uncertain risk of aminoglycoside-induced heari...
8,CPIC_Guideline,"Annotation of CPIC Guideline for amikacin, gen...",amikacin,MT_RNR1,MT-RNR1 827A>G,Normal risk of aminoglycoside-induced hearing ...
9,CPIC_Guideline,"Annotation of CPIC Guideline for amikacin, gen...",amikacin,MT_RNR1,MT-RNR1 839A>G,Uncertain risk of aminoglycoside-induced heari...


In [3]:
print(set(['CYP2C19', 'CYP2D6', 'NUDT15', 'TPMT', 'CYP2C9', 'CFTR', 'CYP2D6', 'G6PD',
           'SLCO1B1', 'UGT1A1', 'DPYD', 'HLA_B', 'HLA_A', 'HLA_B', 'CYP2C9', 'HLA_B',
           'NUDT15', 'CYP3A5', 'IFNL3', 'CYP2C19', 'CYP2B6', 'MT_RNR1', 'CACNA1S', 'RYR1']))

{'DPYD', 'CYP3A5', 'UGT1A1', 'TPMT', 'CACNA1S', 'RYR1', 'HLA_B', 'CYP2C9', 'CFTR', 'IFNL3', 'MT_RNR1', 'CYP2C19', 'CYP2D6', 'NUDT15', 'SLCO1B1', 'HLA_A', 'CYP2B6', 'G6PD'}


In [4]:
df_guideline.assign(
    data_source=["guideline"] * len(df_guideline)
).to_csv("processed/guideline_drug_variant_annotation.csv", index=False)

In [2]:
html = "<h3 id=\"november-2018-update\">November 2018 Update</h3>\n<p>The Royal Dutch Pharmacists Association - Pharmacogenetics Working Group has evaluated therapeutic dose recommendations for aripiprazole based on CYP2D6 genotype. They recommend reducing maximum dose of aripiprazole for CYP2D6 poor metabolizer (PM).</p>\n<p><em>Wording in table taken from the <a href=\"https://api.pharmgkb.org/v1/download/file/attachment/DPWG_November_2018.pdf\">Dutch guidelines November 2018 update</a></em></p>\n<table class=\"table\">\n<thead>\n<tr><th>Allele/Genotype/Phenotype</th><th>Drug</th><th>Description</th><th>Recommendation</th></tr>\n</thead>\n<tbody>\n<tr><td>CYP2D6 UM</td><td>aripiprazole</td><td>-</td><td>NO action is needed for this gene-drug interaction. The genetic variation decreases the plasma concentration of the sum of aripiprazole and the active metabolite dehydroaripiprazole to a limited degree. There is no evidence that this increases the risk of reduced effectiveness.</td></tr>\n<tr><td>CYP2D6 IM</td><td>aripiprazole</td><td>-</td><td>NO action is needed for this gene-drug interaction.The genetic variation increases the plasma concentration of the sum of aripiprazole and the active metabolite dehydroaripiprazole to a limited degree. There is insufficient evidence that this increases the risk of side effects.</td></tr>\n<tr><td>CYP2D6 PM</td><td>aripiprazole</td><td>The risk of side effects is increased. The genetic variation leads to an increase in the sum of the plasma concentrations of aripiprazole and the active metabolite.</td><td>Administer no more than 10 mg/day or 300 mg/month (67-75% of the standard maximum dose of aripiprazole).</td></tr>\n</tbody>\n</table>\n<p><a rel=\"noopener noreferrer\" href=\"https://www.g-standaard.nl/risicoanalyse/B0001543.PDF\" target=\"_blank\">Read for more information about this recommendation</a>, <a rel=\"noopener noreferrer\" href=\"https://www.knmp.nl/producten/gebruiksrecht-g-standaard/medicatiebewaking-g-standaard/background-information-pharmacogenetics\" target=\"_blank\">Read about gene information from DPWG</a></p>\n<h3 id=\"2011-guideline\">2011 Guideline</h3>\n<p>The Royal Dutch Pharmacists Association - Pharmacogenetics Working Group (DPWG) has evaluated therapeutic dose recommendations for aripiprazole based on CYP2D6 genotypes [Article:<a href=\"/pmid/21412232\">21412232</a>].  They recommend reducing maximum dose of aripiprazole for patients carrying poor metabolizer alleles of CYP2D6.</p>\n<table class=\"table\">\n<thead>\n<tr><th>Phenotype (Genotype)</th><th>Therapeutic Dose Recommendation</th><th>Level of Evidence</th><th>Clinical Relevance</th></tr>\n</thead>\n<tbody>\n<tr><td>PM (two inactive (*3-*8, *11-*16, *19-*21, *38, *40, *42) alleles)</td><td>Reduce maximum dose to 10 mg/day (67% of the maximum recommended daily dose).</td><td>Published controlled studies of good quality* relating to phenotyped and/or genotyped patients or healthy volunteers, and having relevant pharmacokinetic or clinical endpoints.</td><td>Clinical effect (S): long-standing discomfort (48-168 hr) without permanent injury e.g. failure of therapy with tricyclic antidepressants, atypical antipsychotic drugs; extrapyramidal side effects; parkinsonism; ADE resulting from increased bioavailability of tricyclic antidepressants, metoprolol, propafenone (central effects e.g. dizziness); INR 4.5-6.0; neutropenia 1.0-1.5x109/l; leucopenia 2.0-3.0x109/l; thrombocytopenia 50-75x109/l.</td></tr>\n<tr><td>IM (two decreased-activity (*9, *10, *17, *29, *36, *41) alleles or carrying one active (*1, *2, *33, *35) and one inactive (*3-*8, *11-*16, *19-*21, *38, *40, *42) allele, or carrying one decreased-activity (*9, *10, *17, *29, *36, *41) allele and one inactive (*3-*8, *11-*16, *19-*21, *38, *40, *42) allele)</td><td>No recommendations.</td><td>Published controlled studies of good quality* relating to phenotyped and/or genotyped patients or healthy volunteers, and having relevant pharmacokinetic or clinical endpoints.</td><td>Minor clinical effect (S): QTc prolongation (&lt;450 ms female, &lt;470 ms male); INR increase &lt; 4.5; Kinetic effect (S)</td></tr>\n<tr><td>UM (a gene duplication in absence of inactive (*3-*8, *11-*16, *19-*21, *38, *40, *42) or decreased-activity (*9, *10, *17, *29, *36, *41) alleles)</td><td>No recommendations.</td><td>--</td><td>--</td></tr>\n</tbody>\n</table>\n<ul>\n<li>*See <a href=\"http://www.pharmgkb.org/page/dpwg\">Methods </a> or [Article:<a href=\"/pmid/18253145\">18253145</a>] for definition of &quot;good quality.&quot;</li>\n<li>S: statistically significant difference.</li>\n<li>Please see attached PDF for detailed information about the evaluated studies: <a href=\"https://api.pharmgkb.org/v1/download/file/attachment/aripiprazole_CYP2D6_271111.pdf\">Aripiprazole CYP2D6 </a></li>\n</ul>\n"

In [3]:
soup = BeautifulSoup(html, "html.parser")

In [9]:
list_header = []
header = soup.find_all("table")[0].find("tr")
for items in header:
    try:
        list_header.append(items.get_text())
    except:
        continue

In [10]:
list_header

['Allele/Genotype/Phenotype', 'Drug', 'Description', 'Recommendation']

In [11]:
HTML_data = soup.find_all("table")[0].find_all("tr")[1:]
data = []

for element in HTML_data:
    sub_data = []
    for sub_element in element:
        try:
            sub_data.append(sub_element.get_text())
        except:
            continue
    data.append(sub_data)

In [12]:
data

[['CYP2D6 UM',
  'aripiprazole',
  '-',
  'NO action is needed for this gene-drug interaction. The genetic variation decreases the plasma concentration of the sum of aripiprazole and the active metabolite dehydroaripiprazole to a limited degree. There is no evidence that this increases the risk of reduced effectiveness.'],
 ['CYP2D6 IM',
  'aripiprazole',
  '-',
  'NO action is needed for this gene-drug interaction.The genetic variation increases the plasma concentration of the sum of aripiprazole and the active metabolite dehydroaripiprazole to a limited degree. There is insufficient evidence that this increases the risk of side effects.'],
 ['CYP2D6 PM',
  'aripiprazole',
  'The risk of side effects is increased. The genetic variation leads to an increase in the sum of the plasma concentrations of aripiprazole and the active metabolite.',
  'Administer no more than 10 mg/day or 300 mg/month (67-75% of the standard maximum dose of aripiprazole).']]

In [15]:
pd.DataFrame(data = data, columns = list_header)

,Allele/Genotype/Phenotype,Drug,Description,Recommendation
0,CYP2D6 UM,aripiprazole,-,NO action is needed for this gene-drug interac...
1,CYP2D6 IM,aripiprazole,-,NO action is needed for this gene-drug interac...
2,CYP2D6 PM,aripiprazole,The risk of side effects is increased. The gen...,Administer no more than 10 mg/day or 300 mg/mo...


In [2]:
def parse_tables(html, gene, drug, organization, link):
    # save columns which contain these pharses
    column_template = ["phenotype", "diplotype", "genotype", "implication", "description", "recommendation", "dosing"]
    soup = BeautifulSoup(html, "html.parser")
    tables = soup.find_all("table")
    titles = soup.find_all("h3")
    df_list = []
    for i in range(len(tables)):
        try:
            table = tables[i]
            title = titles[i].get_text()
        except Exception as e:
            print("missing title, gene {}, drug {}, organization {}".format(gene, drug, organization))
            continue
        list_header = []
        header = table.find("tr")
        for items in header:
            try:
                t = items.get_text()
                if t == "\n":
                    continue
                list_header.append(t)
            except:
                continue

        HTML_data = table.find_all("tr")[1:]
        data = []
        for element in HTML_data:
            sub_data = []
            for sub_element in element:
                try:
                    t = sub_element.get_text()
                    if t == "\n":
                        continue
                    sub_data.append(t)
                except:
                    continue
            data.append(sub_data)

        try:
            df_tmp = pd.DataFrame(data=data, columns=list_header)
            save_column = []
            for column in df_tmp.columns:
                if any([True if x in column.lower() else False for x in column_template]):
                    save_column.append(column)
            df_tmp = df_tmp[save_column]
            
            df_tmp = df_tmp.assign(
                gene=[gene] * len(df_tmp)
            ).assign(
                drug=[drug] * len(df_tmp)
            ).assign(
                organization=[organization] * len(df_tmp)
            ).assign(
                title=[title] * len(df_tmp)
            ).assign(
                link=[link] * len(df_tmp)
            )
            df_list.append(df_tmp)
        except Exception as e:
            print("parsing html failed, gene {}, drug {}, organization {}".format(gene, drug, organization))
            continue
    return df_list
    

In [3]:
tables = []
folder = "guideline_annotation"
for path in os.listdir(folder):
    if path.endswith(".json") and "Annotation_of_" in path:
        with open(os.path.join(folder, path), "r", encoding="utf-8") as f:
            guideline_dict = json.load(f)
            
        path = path.replace("Annotation_of_", "").replace(".json", "")
        try:
            guideline, drug_gene = path.split("_for_")
        except:
            continue
        drugs, genes = drug_gene.split("_and_")
        html = guideline_dict["guideline"]["textMarkdown"]["html"]
        link = guideline_dict["guideline"]["@id"]
        
        tables.extend(parse_tables(guideline_dict["guideline"]["textMarkdown"]["html"], 
                                   gene=genes, drug=drugs, organization=guideline, link=link))
        

missing title, gene CYP2D6, drug tamoxifen, organization CPNDS_Guideline
missing title, gene CYP2D6, drug haloperidol, organization DPWG_Guideline
missing title, gene CYP2D6, drug imipramine, organization DPWG_Guideline
missing title, gene CYP2D6, drug metoprolol, organization DPWG_Guideline
missing title, gene CYP2D6, drug nortriptyline, organization DPWG_Guideline
missing title, gene CYP2D6, drug paroxetine, organization DPWG_Guideline
missing title, gene CYP2D6, drug propafenone, organization DPWG_Guideline
missing title, gene CYP2C9_VKORC1, drug acenocoumarol_fluindione_warfarin, organization PRO_Guideline


In [8]:
["phenotype", "diplotype", "genotype", "implication", "description", "recommendation"]

['phenotype',
 'diplotype',
 'genotype',
 'implication',
 'description',
 'recommendation']

In [5]:
phenotype_category_list = []
phenotype_list = []
diplotype_list = []
genotype_list = []
implication_list = []
description_list = []
recommendation_list = []
dosing_list = []
gene_list = []
drug_list = []
organization_list = []
title_list = []
link_list = []
for table in tables:
    phenotype_column = ""
    diplotype_column = ""
    genotype_column = ""
    implication_column = ""
    description_column = ""
    recommendation_column = ""
    dosing_column = ""
    for col in table.columns:
        if "phenotype" in col.lower():
            phenotype_column = col
        if "diplotype" in col.lower():
            diplotype_column = col
        if "genotype" in col.lower():
            genotype_column = col
        if "implication" in col.lower():
            implication_column = col
        if "description" in col.lower():
            description_column = col
        if "recommendation" in col.lower() and "classification" not in col.lower():
            recommendation_column = col
        if "dosing" in col.lower():
            dosing_column = col
            
    for index, row in table.iterrows():
        if phenotype_column != "":
            phenotype = row[phenotype_column]
        else:
            continue
        
        if diplotype_column != "":
            diplotype = row[diplotype_column]
        else:
            diplotype = ""
        
        if genotype_column != "":
            genotype = row[genotype_column]
        else:
            genotype = ""

        if implication_column != "":
            implication = row[implication_column]
        else:
            implication = ""
            
        if description_column != "":
            description = row[description_column]
        else:
            description = ""
            
        if recommendation_column != "":
            recommendation = row[recommendation_column]
        else:
            recommendation = ""
            
        if dosing_column != "":
            dosing = row[dosing_column]
        else:
            dosing = ""
        
        gene = row["gene"]
        drug = row["drug"]
        organization = row["organization"]
        title = row["title"]

        if any([True if x in phenotype.lower() and "meta" in phenotype.lower() else False \
                for x in ["intermediate", "normal", "poor", "rapid", "ultrarapid", "extensive"]]) or \
           any(True if x in phenotype else False \
                for x in ["IM", "PM", "UM", "NM", "RM", "EM"]):
            pass
        else:
            continue
            
        if genotype == "" and implication == "" and description == "" and recommendation == "" and dosing == "":
            continue
            
        if "_" in gene:
            genes = gene.split("_")
            filtered_gene = list(filter(lambda x: x in phenotype, genes))
            if len(filtered_gene) > 0:
                gene = filtered_gene[0] 
                
        if "Drug" in table.columns and row["Drug"] != "":
            drug = row["Drug"]
        elif "_" in drug:
            drugs = drug.split("_")
            filtered_drug = list(filter(lambda x: x in phenotype or x in recommendation or x in implication, drugs))
            if len(filtered_drug) > 0:
                drug = filtered_drug[0] 
        
        phenotype_category = ""
        if "poor" in phenotype.lower() or "PM" in phenotype:
            phenotype_category = "Poor Metabolizer"
        elif "intermediate" in phenotype.lower() or "IM" in phenotype:
            phenotype_category = "Intermediate metabolizer"
        elif "normal" in phenotype.lower() or "NM" in phenotype:
            phenotype_category = "Normal metabolizer"
        elif ("rapid" in phenotype.lower() and "ultra" not in phenotype.lower()) or "RM" in phenotype:
            phenotype_category = "Rapid metabolizer"
        elif ("rapid" in phenotype.lower() and "ultra" in phenotype.lower()) or "UM" in phenotype:
            phenotype_category = "Ultrarapid metabolizer"
        elif "extensive" in phenotype.lower() or "EM" in phenotype:
            phenotype_category = "Extensive metabolizer"
            
        phenotype_category_list.append(phenotype_category)
        phenotype_list.append(phenotype)
        diplotype_list.append(diplotype)
        genotype_list.append(genotype)
        implication_list.append(implication)
        description_list.append(description)
        recommendation_list.append(recommendation)
        dosing_list.append(dosing)
        gene_list.append(gene)
        drug_list.append(drug)
        organization_list.append(organization)
        title_list.append(title)
        link_list.append(row["link"])
            
df_pheno_drug = pd.DataFrame({
    "phenotype_category": phenotype_category_list,
    "phenotype": phenotype_list,
    "diplotype": diplotype_list,
    "genotype": genotype_list,
    "implication": implication_list,
    "description": description_list,
    "recommendation": recommendation_list,
    "dosing": dosing_list,
    "gene": gene_list,
    "drug": drug_list,
    "organization": organization_list,
    "title": title_list,
    "link": link_list,
    "data_source": ["guildeline"] * len(link_list)
})

In [16]:
# for diplotype
phenotype_category_list = [] 
phenotype_list = []
diplotype_list = []
genotype_list = []
implication_list = []
description_list = []
recommendation_list = []
gene_list = []
drug_list = []
organization_list = []
title_list = []
link_list = []

for index, row in df_pheno_drug.iterrows():
    diplotype = row["diplotype"]
    diplotypes = re.split(",|;| ", diplotype)
    diplotypes = list(filter(lambda x: x != "" and x.count("*") == 2, diplotypes))
    if len(diplotypes) == 0:
        continue
        
    for dip in diplotypes:
        dip_name = "{} {}".format(row["gene"], dip)
        recommendation = row["dosing"] if row["dosing"] != "" else row["recommendation"]
        phenotype_category_list.append(row["phenotype_category"])
        phenotype_list.append(row["phenotype"])
        diplotype_list.append(dip_name)
        genotype_list.append(row["genotype"])
        implication_list.append(row["implication"])
        description_list.append(row["description"])
        recommendation_list.append(row["recommendation"])
        gene_list.append(row["gene"])
        drug_list.append(row["drug"])
        organization_list.append(row["organization"])
        title_list.append(row["title"])
        link_list.append(row["link"])
        
pd.DataFrame({
    "phenotype_category": phenotype_category_list,
    "phenotype": phenotype_list,
    "diplotype": diplotype_list,
    "genotype": genotype_list,
    "implication": implication_list,
    "description": description_list,
    "recommendation": recommendation_list,
    "gene": gene_list,
    "drug": drug_list,
    "organization": organization_list,
    "title": title_list,
    "link": link_list,
    "data_source": ["guildeline"] * len(link_list)
}).to_csv("processed/diplotype_drug_relation.csv", index=False)

df_pheno_drug.to_csv("processed/phenotype_drug_relation.csv", index=False)

In [218]:
len(df_pheno_drug)

473

In [229]:
len(link_list)

1892

In [5]:
pd.DataFrame({
    "node": ["diplotype", "chemical", "gene", "haplotype", "rsID", "nmpa_drug", "total"],
    "count": [16574, 932, 2013, 1526, 6412, 33608, 61065]
})

,node,count
0,diplotype,16574
1,chemical,932
2,gene,2013
3,haplotype,1526
4,rsID,6412
5,nmpa_drug,33608
6,total,61065


In [4]:
pd.DataFrame({
    "edge": ["clinical_annotation", "diplotype_consist_of", "diplotype_metabolizer", "drug_label",
             "guideline_annotation", "haplotype_rsID_related", "mutation_at", "research_annotation",
             "drug_consist_of", "total"],
    "count": [8318, 32420, 10291, 29628, 5797, 785, 8183, 4291, 33608, 133321]
})

,edge,count
0,clinical_annotation,8318
1,diplotype_consist_of,32420
2,diplotype_metabolizer,10291
3,drug_label,29628
4,guideline_annotation,5797
5,haplotype_rsID_related,785
6,mutation_at,8183
7,research_annotation,4291
8,drug_consist_of,33608
9,total,133321


In [5]:
df_pheno = pd.read_csv("processed/phenotype_drug_relation_no_metabolizer.csv").fillna("")
df_pheno

,phenotype_category,phenotype,diplotype,genotype,implication,description,recommendation,dosing,gene,drug,organization,title,link,data_source,update_date,variant
0,,Very low risk of hypersensitivity (constitutes...,*X/*X c,"Absence of *57:01 alleles (reported as ""negati...",Low or reduced risk of abacavir hypersensitivity,,Use abacavir per standard dosing guidelines,,HLA_B,abacavir,CPIC_Guideline,May 2014 Update,https://pharmgkb.org/guidelineAnnotation/PA166...,guildeline,2021-10-05,
1,,High risk of hypersensitivity (~6% of patients),*57:01/*X c *57:01/*57:01,Presence of at least one *57:01 allele (report...,Significantly increased risk of abacavir hyper...,,Abacavir is not recommended,,HLA_B,abacavir,CPIC_Guideline,May 2014 Update,https://pharmgkb.org/guidelineAnnotation/PA166...,guildeline,2021-10-05,
2,,Low or reduced risk of allopurinol SCAR,*X/*X a,"Absence of *58:01 alleles (reported as ""negati...",Low or reduced risk of allopurinol SCAR,,Use allopurinol per standard dosing guidelines,,HLA_B,allopurinol,CPIC_Guideline,January 2016 Update,https://pharmgkb.org/guidelineAnnotation/PA166...,guildeline,2021-10-05,
3,,Significantly increased risk of allopurinol SCAR,*58:01/*X a *58:01/*58:01,Presence of at least one *58:01 allele (report...,Significantly increased risk of allopurinol SCAR,,Allopurinol is contraindicated,,HLA_B,allopurinol,CPIC_Guideline,January 2016 Update,https://pharmgkb.org/guidelineAnnotation/PA166...,guildeline,2021-10-05,
4,,Homozygous for alleles other than HLA-B*15:02 ...,*X/*X b*Y/*Y c,Non-carrier of HLA-B*15:02 or HLA-A*31:01. No...,Normal or reduced risk of carbamazepine-induce...,,Use carbamazepine per standard dosing guidelin...,,HLA,carbamazepine,CPIC_Guideline,December 2017,https://pharmgkb.org/guidelineAnnotation/PA166...,guildeline,2021-10-05,
5,,Heterozygote or homozygous for HLA-A*31:01 and...,*X/*X b*31:01/*Y c*31:01/*31:01,Carrier of HLA-A*31:01. One or two *31:01 all...,Increased risk of carbamazepine-induced SJS/TE...,,A. If patient is carbamazepine-naive and alter...,,HLA,carbamazepine,CPIC_Guideline,December 2017,https://pharmgkb.org/guidelineAnnotation/PA166...,guildeline,2021-10-05,
6,,Heterozygote or homozygous for HLA-B*15:02 and...,*15:02/*X b*15:02/*15:02,Carrier of HLA-B*15:02. One or two *15:02 all...,Increased risk of carbamazepine-induced SJS/TEN,,"A. If patient is carbamazepine-naive, do not u...",,HLA,carbamazepine,CPIC_Guideline,December 2017,https://pharmgkb.org/guidelineAnnotation/PA166...,guildeline,2021-10-05,
7,,Malignant Hyperthermia susceptible,,An individual heterozygousb for a RYR1 or CACN...,Individuals are at increased risk of developin...,,Halogenated volatile anesthetics or depolarizi...,Halogenated volatile anesthetics or depolarizi...,CACNA1S_RYR1,desflurane,CPIC_Guideline,November 2018,https://pharmgkb.org/guidelineAnnotation/PA166...,guildeline,2021-10-05,"RYR1 c.103T>C; p.(Cys35Arg), RYR1 c.130C>T; p...."
8,,Uncertain susceptibility,,An individual negative for a RYR1 or CACNA1S m...,These results do not eliminate the chance that...,,"Clinical findings, family history, further gen...","Clinical findings, family history, further gen...",CACNA1S_RYR1,desflurane,CPIC_Guideline,November 2018,https://pharmgkb.org/guidelineAnnotation/PA166...,guildeline,2021-10-05,
9,,Homozygous for alleles other than HLA-B*15:02,*X/*X b,Non-carrier of HLA-B*15:02. No HLA-B*15:02 al...,Normal or reduced risk of oxcarbazepine-induce...,,Use oxcarbazepine per standard dosing guidelines.,,HLA,oxcarbazepine,CPIC_Guideline,December 2017,https://pharmgkb.org/guidelineAnnotation/PA166...,guildeline,2021-10-05,


In [28]:
haplotype_record_list = []
diplotype_record_list = []

for index, row in df_pheno.iterrows():
    diplotype_list = row["diplotype"].replace("a", "").replace("b", "").replace("c", "").replace("d", "").split(" ")
    diplotype_list = list(filter(lambda x: x != "" and "/" in x  and x != "/", [x.replace(",", "").replace(";", "").strip() for x in diplotype_list]))
    variant_list = list(filter(lambda x: x != "", [x.split(";")[0].strip() for x in row["variant"].split(",")]))
    gene = row["gene"].replace("_", "-")
    for diplo in diplotype_list:
        diplotype_record_list.append(
            ("{} {}".format(gene, diplo), row["phenotype_category"], row["phenotype"], row["genotype"],
             row["implication"], row["description"], row["recommendation"], row["dosing"], row["drug"],
             row["organization"], row["title"], row["link"], row["data_source"], row["update_date"])
        )
    for var in variant_list:
        haplotype_record_list.append(
            (var, row["phenotype_category"], row["phenotype"], row["genotype"],
             row["implication"], row["description"], row["recommendation"], row["dosing"], row["drug"],
             row["organization"], row["title"], row["link"], row["data_source"], row["update_date"])
        )

In [30]:
df_haplotype_guideline = pd.DataFrame(
    haplotype_record_list,
    columns=["haplotype", "phenotype_category", "phenotype", "genotype",
             "implication", "description", "recommendation", "dosing", "drug",
             "organization", "title", "link", "data_source", "update_date"])

df_diplotype_guideline = pd.DataFrame(
    diplotype_record_list,
    columns=["diplotype", "phenotype_category", "phenotype", "genotype",
             "implication", "description", "recommendation", "dosing", "drug",
             "organization", "title", "link", "data_source", "update_date"])

In [33]:
df_haplotype_guideline

,haplotype,phenotype_category,phenotype,genotype,implication,description,recommendation,dosing,drug,organization,title,link,data_source,update_date
0,RYR1 c.103T>C,,Malignant Hyperthermia susceptible,An individual heterozygousb for a RYR1 or CACN...,Individuals are at increased risk of developin...,,Halogenated volatile anesthetics or depolarizi...,Halogenated volatile anesthetics or depolarizi...,desflurane,CPIC_Guideline,November 2018,https://pharmgkb.org/guidelineAnnotation/PA166...,guildeline,2021-10-05
1,RYR1 c.130C>T,,Malignant Hyperthermia susceptible,An individual heterozygousb for a RYR1 or CACN...,Individuals are at increased risk of developin...,,Halogenated volatile anesthetics or depolarizi...,Halogenated volatile anesthetics or depolarizi...,desflurane,CPIC_Guideline,November 2018,https://pharmgkb.org/guidelineAnnotation/PA166...,guildeline,2021-10-05
2,RYR1 c.487C>T,,Malignant Hyperthermia susceptible,An individual heterozygousb for a RYR1 or CACN...,Individuals are at increased risk of developin...,,Halogenated volatile anesthetics or depolarizi...,Halogenated volatile anesthetics or depolarizi...,desflurane,CPIC_Guideline,November 2018,https://pharmgkb.org/guidelineAnnotation/PA166...,guildeline,2021-10-05
3,RYR1 c.488G>T,,Malignant Hyperthermia susceptible,An individual heterozygousb for a RYR1 or CACN...,Individuals are at increased risk of developin...,,Halogenated volatile anesthetics or depolarizi...,Halogenated volatile anesthetics or depolarizi...,desflurane,CPIC_Guideline,November 2018,https://pharmgkb.org/guidelineAnnotation/PA166...,guildeline,2021-10-05
4,RYR1 c.742G>A/C,,Malignant Hyperthermia susceptible,An individual heterozygousb for a RYR1 or CACN...,Individuals are at increased risk of developin...,,Halogenated volatile anesthetics or depolarizi...,Halogenated volatile anesthetics or depolarizi...,desflurane,CPIC_Guideline,November 2018,https://pharmgkb.org/guidelineAnnotation/PA166...,guildeline,2021-10-05
5,RYR1 c.982C>T,,Malignant Hyperthermia susceptible,An individual heterozygousb for a RYR1 or CACN...,Individuals are at increased risk of developin...,,Halogenated volatile anesthetics or depolarizi...,Halogenated volatile anesthetics or depolarizi...,desflurane,CPIC_Guideline,November 2018,https://pharmgkb.org/guidelineAnnotation/PA166...,guildeline,2021-10-05
6,RYR1 c.1021G>C,,Malignant Hyperthermia susceptible,An individual heterozygousb for a RYR1 or CACN...,Individuals are at increased risk of developin...,,Halogenated volatile anesthetics or depolarizi...,Halogenated volatile anesthetics or depolarizi...,desflurane,CPIC_Guideline,November 2018,https://pharmgkb.org/guidelineAnnotation/PA166...,guildeline,2021-10-05
7,RYR1 c.1021G>A,,Malignant Hyperthermia susceptible,An individual heterozygousb for a RYR1 or CACN...,Individuals are at increased risk of developin...,,Halogenated volatile anesthetics or depolarizi...,Halogenated volatile anesthetics or depolarizi...,desflurane,CPIC_Guideline,November 2018,https://pharmgkb.org/guidelineAnnotation/PA166...,guildeline,2021-10-05
8,RYR1 c.1201C>T,,Malignant Hyperthermia susceptible,An individual heterozygousb for a RYR1 or CACN...,Individuals are at increased risk of developin...,,Halogenated volatile anesthetics or depolarizi...,Halogenated volatile anesthetics or depolarizi...,desflurane,CPIC_Guideline,November 2018,https://pharmgkb.org/guidelineAnnotation/PA166...,guildeline,2021-10-05
9,RYR1 c.1209C>G,,Malignant Hyperthermia susceptible,An individual heterozygousb for a RYR1 or CACN...,Individuals are at increased risk of developin...,,Halogenated volatile anesthetics or depolarizi...,Halogenated volatile anesthetics or depolarizi...,desflurane,CPIC_Guideline,November 2018,https://pharmgkb.org/guidelineAnnotation/PA166...,guildeline,2021-10-05
